In [12]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
import spotipy
from sklearn.manifold import TSNE
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import plotly.express as px


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import string
import re


load_dotenv()
SPOTIFY_KEY1 = os.getenv('SPOTIFY_KEY1')
SPOTIFY_KEY2 = os.getenv('SPOTIFY_KEY2')
SPOTIFY_DATA = os.getenv('SPOTIFY_DATA')
TCC_DATA = os.getenv('TCC_DATA')

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(SPOTIFY_KEY1,
                                                                              SPOTIFY_KEY2))

In [21]:
df = pd.read_csv(SPOTIFY_DATA)
df.sort_values('popularity', ascending=False)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
19611,0.1450,2020,0.4010,"['Bad Bunny', 'Jhay Cortez']",0.731,205090,0.5730,1,47EiUVwUp4C9fGccaPuUCS,0.000052,4,0.1130,-10.059,0,Dakiti,100,2020-10-30,0.0544,109.928
19606,0.7560,2020,0.2210,"['24kGoldn', 'iann dior']",0.700,140526,0.7220,1,3tjFYV6RSFtuktYl3ZtYcq,0.000000,7,0.2720,-3.558,0,Mood (feat. iann dior),99,2020-07-24,0.0369,90.989
19618,0.7370,2020,0.0112,['BTS'],0.746,199054,0.7650,0,0t1kP63rueHleOhQkYSXFY,0.000000,6,0.0936,-4.410,0,Dynamite,97,2020-08-28,0.0993,114.044
19608,0.3570,2020,0.0194,"['Cardi B', 'Megan Thee Stallion']",0.935,187541,0.4540,1,4Oun2ylbjFKMPTiaSbbCih,0.000000,1,0.0824,-7.509,1,WAP (feat. Megan Thee Stallion),96,2020-08-07,0.3750,133.073
19610,0.6820,2020,0.4680,['Ariana Grande'],0.737,172325,0.8020,1,35mvY5S1H3J2QZyna3TFe0,0.000000,0,0.0931,-4.771,1,positions,96,2020-10-30,0.0878,144.015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109919,0.0696,1945,0.9920,"['Modest Mussorgsky', 'William Kapell']",0.224,101427,0.0082,0,3X2WPBNvwLXQ5bxop3ZeFG,0.865000,6,0.0979,-31.113,1,Pictures at an Exhibition: Con mortuis in ling...,0,1945,0.0350,83.252
109920,0.4910,1945,0.1530,['Эрих Мария Ремарк'],0.700,114900,0.1760,1,3XQuNp2KXBAZWwyM5DeWds,0.000036,0,0.1050,-19.316,1,Часть 38.2 - Триумфальная арка,0,1945,0.9120,99.996
109921,0.4440,1945,0.9770,['Charlie'],0.517,170910,0.2540,0,3XsEvDAb1fNAQ1kejK0KLz,0.000189,2,0.2740,-9.750,1,Aai Musibat Sar Pe Bhaari,0,1945-12-31,0.1200,124.506
109922,0.6120,1945,0.1610,['Эрих Мария Ремарк'],0.699,105778,0.2380,1,3XwNVvuxklK3yXZWDUrEVE,0.000000,5,0.1070,-20.042,0,Часть 14.3 & Часть 15.1 - Триумфальная арка,0,1945,0.9130,103.594


In [17]:
# check for nulls

df.isnull().sum()

valence             0
year                0
acousticness        0
artists             0
danceability        0
duration_ms         0
energy              0
explicit            0
id                  0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
name                0
popularity          0
release_date        0
speechiness         0
tempo               0
dtype: int64

In [18]:
# check for dupes
df.duplicated().sum()

0

In [19]:
# no dupes or nulls!
df.shape

(170653, 19)

In [23]:
# drop unnecessary columns
df.drop("release_date", axis = 1)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo
0,0.0594,1921,0.98200,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.6650,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,0.0366,80.954
1,0.9630,1921,0.73200,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.1600,-12.441,1,Clancy Lowered the Boom,5,0.4150,60.936
2,0.0394,1921,0.96100,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.1010,-14.850,1,Gati Bali,5,0.0339,110.339
3,0.1650,1921,0.96700,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.3810,-9.316,1,Danny Boy,3,0.0354,100.109
4,0.2530,1921,0.95700,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.2290,-10.096,1,When Irish Eyes Are Smiling,2,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170648,0.6080,2020,0.08460,"['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...",0.786,301714,0.808,0,0KkIkfsLEJbrcIhYsCL7L5,0.000289,7,0.0822,-3.702,1,China,72,0.0881,105.029
170649,0.7340,2020,0.20600,['Ashnikko'],0.717,150654,0.753,0,0OStKKAuXlxA0fMH54Qs6E,0.000000,7,0.1010,-6.020,1,Halloweenie III: Seven Days,68,0.0605,137.936
170650,0.6370,2020,0.10100,['MAMAMOO'],0.634,211280,0.858,0,4BZXVFYCb76Q0Klojq4piV,0.000009,4,0.2580,-2.226,0,AYA,76,0.0809,91.688
170651,0.1950,2020,0.00998,['Eminem'],0.671,337147,0.623,1,5SiZJoLXp3WOl3J4C8IK0d,0.000008,2,0.6430,-7.161,1,Darkness,70,0.3080,75.055


#### Valence: Measure from 0.0 to 1.0 that describes positivity in tracks.
#### Year: The year in which the track was released.
#### Acousticness: A confidence measure from 0.0 to 1.0 of whether the track is acoustic, higher value is better confidence.
#### Artists: Names of the Artists.
#### Danceability: Describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
#### Duration: Length of song in microseconds
#### Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
#### Explicit: Value 1 = Explicit, Value 0 = Not Explicit
#### id: Song ID via Spotify
#### Instrumentalness: Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
#### The estimated overall key of the track. Integers map to pitches using standard Pitch Class notation . E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.
#### Loudness: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.
#### Mode: Indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.
#### Popularity: The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time.
#### Speechiness: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
#### Tempo: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.

In [26]:
# feature extractions
features = ['valence', 'acousticness', 'danceability', 'energy', 'explicit', 'instrumentalness', 'key', 'loudness', 'mode', 'speechiness', 'tempo' ]

In [27]:
df[features]

,valence,acousticness,danceability,energy,explicit,instrumentalness,key,loudness,mode,speechiness,tempo
0,0.0594,0.98200,0.279,0.211,0,0.878000,10,-20.096,1,0.0366,80.954
1,0.9630,0.73200,0.819,0.341,0,0.000000,7,-12.441,1,0.4150,60.936
2,0.0394,0.96100,0.328,0.166,0,0.913000,3,-14.850,1,0.0339,110.339
3,0.1650,0.96700,0.275,0.309,0,0.000028,5,-9.316,1,0.0354,100.109
4,0.2530,0.95700,0.418,0.193,0,0.000002,3,-10.096,1,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...
170648,0.6080,0.08460,0.786,0.808,0,0.000289,7,-3.702,1,0.0881,105.029
170649,0.7340,0.20600,0.717,0.753,0,0.000000,7,-6.020,1,0.0605,137.936
170650,0.6370,0.10100,0.634,0.858,0,0.000009,4,-2.226,0,0.0809,91.688
170651,0.1950,0.00998,0.671,0.623,1,0.000008,2,-7.161,1,0.3080,75.055


In [29]:
def getid(artist_name, song_name):
    artistname = artist_name.strip("['']")
    results = spotify.search(q="track" + song_name + "artist" + artistname, type="track", limit=1)
    if "tracks" in results and "items" in results["tracks"]:
        return results["tracks"]["items"][0]["id"]
    else:
        print("Track not found.")
        return None


def name_lookup(name, df):
    formatting_input = "['{}']".format(string.capwords(name))
    lookup = df[df['artists'].str.lower() == formatting_input.lower()]

    if not lookup.empty:
        return formatting_input
    else:
        print("Artist not found")
        return False


def song_lookup(song, df):
    lookup = df[df['name'].str.lower() == song.lower()]

    if not lookup.empty:
        return string.capwords(song)
    else:
        print("Song not found.")
        return False


def print_info(track):
    print('\nArtist: ' + track['artists'][0]['name'])
    print('Track: ' + track['name'])
    print('Album: ' + track['album']['name'])
    AP = track['preview_url']
    if AP is None:
        print("No Audio Preview available")
    else:
        print('Audio Preview: ' + track['preview_url'])
    print('Cover Art: ' + track['album']['images'][0]['url'])

In [32]:
found = False
while not found:
    artist_name = input("Enter artist name: ")
    if name_lookup(artist_name, df):
        found = True
        artist_name=name_lookup(artist_name,df)

found = False
while not found:
    song_name = input("Enter song name: ")
    if song_lookup(song_name, df):
        found = True
        song_name = song_lookup(song_name, df)

track_id = getid(artist_name, song_name)
track = spotify.track(track_id)
print_info(track)

Enter artist name: John Lennon
Enter song name: Imagine

Artist: John Lennon Experience
Track: Imagine
Album: Imagine - Remembering Lennon 40th Anniversary
Audio Preview: https://p.scdn.co/mp3-preview/6b0f7cb7fd5888a5600d47e7ba9b8c9445c9c4c5?cid=798c070d2d5e4ab98b36353e469dba19
Cover Art: https://i.scdn.co/image/ab67616d0000b273fe84759836417d9c41cb9a16


In [35]:
spotify_track = input('Enter Spotify track link: ')
song_URI = spotify_track.split("/")[-1].split("?")[0]
print(song_URI)

Enter Spotify track link: https://open.spotify.com/track/4y5bvROuBDPr5fuwXbIBZR?si=feb1802275c34904
4y5bvROuBDPr5fuwXbIBZR


In [48]:
song_info = spotify.audio_features(song_URI)
track = spotify.track(song_URI)
print(song_info)
print(track['artists'][0]['name'])
print(track['name'])

[{'danceability': 0.811, 'energy': 0.719, 'key': 9, 'loudness': -6.553, 'mode': 1, 'speechiness': 0.0497, 'acousticness': 0.0129, 'instrumentalness': 1.36e-05, 'liveness': 0.0742, 'valence': 0.865, 'tempo': 103.979, 'type': 'audio_features', 'id': '4y5bvROuBDPr5fuwXbIBZR', 'uri': 'spotify:track:4y5bvROuBDPr5fuwXbIBZR', 'track_href': 'https://api.spotify.com/v1/tracks/4y5bvROuBDPr5fuwXbIBZR', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4y5bvROuBDPr5fuwXbIBZR', 'duration_ms': 222400, 'time_signature': 4}]
Taylor Swift
Paper Rings


In [28]:
X = df[features]


user_profile = pd.Series(user_profile, index=features)
similarity = cosine_similarity(X, user_profile.values.reshape(1, -1))



NameError: name 'user_profile' is not defined